# Parsing ImpactT.in

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [2]:
from impact.parsers import *

In [3]:
from impact.lattice import *

In [4]:
with open('../templates/lcls_injector/ImpactT.in', 'r') as f:    
    data = f.read()
    LINES = data.split('\n')

In [5]:
parse_header(LINES)

{'Npcol': 1,
 'Nprow': 1,
 'Dt': 5e-13,
 'Ntstep': 1000000,
 'Nbunch': 1,
 'Dim': 6,
 'Np': 300004,
 'Flagmap': 1,
 'Flagerr': 0,
 'Flagdiag': 2,
 'Flagimg': 1,
 'Zimage': 0.02,
 'Nx': 64,
 'Ny': 64,
 'Nz': 128,
 'Flagbc': 1,
 'Xrad': 0.015,
 'Yrad': 0.015,
 'Perdlen': 45.0,
 'Flagdist': 16,
 'Rstartflg': 0,
 'Flagsbstp': 0,
 'Nemission': 400,
 'Temission': 1.4e-11,
 'sigx(m)': 0.0006,
 'sigpx': 0.0,
 'muxpx': 0.0,
 'xscale': 1.0,
 'pxscale': 1.0,
 'xmu1(m)': 0.0,
 'xmu2': 0.0,
 'sigy(m)': 0.0006,
 'sigpy': 0.0,
 'muxpy': 0.0,
 'yscale': 1.0,
 'pyscale': 1.0,
 'ymu1(m)': 0.0,
 'ymu2': 0.0,
 'sigz(m)': 1.27e-06,
 'sigpz': 0.0,
 'muxpz': 0.0,
 'zscale': 1.0,
 'pzscale': 1.0,
 'zmu1(m)': 0.0,
 'zmu2': 0.0,
 'Bcurr': 0.714,
 'Bkenergy': 1.0,
 'Bmass': 511005.0,
 'Bcharge': -1.0,
 'Bfreq': 2856000000.0,
 'Tini': 0.0}

In [7]:
# Find index of the line where the lattice starts
IX_LATTICE = ix_lattice(LINES)

# Gather lattice lines
LATLINES = LINES[IX_LATTICE:]

In [8]:
# Summary of elements in this lattice
def myf(t): return [l for l in LATLINES if parse_type(l) == t ] 
quads = myf('quadrupole')
drifts = myf('drift')
solrfs = myf('solrf')
dipoles = myf('dipole')

alltypes = set([parse_type(l) for l in LATLINES])
print(alltypes)

for t in ele_type.values():
    l = len(myf(t))
    if l > 0:
        print(l, t)

{'quadrupole', 'write_beam', 'stop', 'dipole', 'comment', 'offset_beam', 'spacecharge', 'wakefield', 'solrf', 'drift', 'write_beam_for_restart', 'change_timestep'}
56 drift
23 quadrupole
6 dipole
26 solrf
9 offset_beam
5 write_beam
1 write_beam_for_restart
1 change_timestep
6 wakefield
2 spacecharge
2 stop


# Individual element parsing

In [9]:
parse_drift(drifts[0])

{'zedge': 1.387, 'radius': 0.15}

In [10]:
parse_quadrupole(quads[0])

{'zedge': 4.752856,
 'b1_gradient': 1.8524,
 'L_effective': 0.108,
 'radius': 0.016,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'x_rotation': 0.0,
 'y_rotation': 0.0,
 'z_rotation': 0.0}

In [11]:
parse_solrf(solrfs[0])

{'zedge': 0.0,
 'rf_field_scale': 47000000.0,
 'rf_frequency': 2856000000.0,
 'theta0_deg': 293.54,
 'filename': 'rfdata201',
 'radius': 0.15,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'x_rotation': 0.0,
 'y_rotation': 0.0,
 'z_rotation': 0.0,
 'solenoid_field_scale': 0.0}

In [12]:
parse_dipole(dipoles[0])

{'zedge': 16.988375,
 'b_field_x': 0.0,
 'b_field': -0.6482593,
 'filename': 'rfdata316',
 'half_gap': 0.015}

In [13]:
parse_spacecharge(myf('spacecharge')[0])

{'s': 31.273, 'is_on': 'False'}

# Full lattice parsing into element dicts

In [14]:
from impact.lattice import ele_str

In [15]:
# This parses all lines. 
eles = parse_lattice(LATLINES)

In [35]:
for e in eles:
    print(ele_line(e))

0.0 1 1 -4 0.0 0.0 0.0 5e-13 /
!**************************1.5 cell gun ************************
0.15 0 0 105 0.0 47000000.0 2856000000.0 293.54 201 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /
0.49308 0 0 105 0.0 0.0 0.0 0.0 102 0.15 0.0 0.0 0.0 0.0 0.0 0.243 /
!0.163  10 20 105 0.0 0.0 0.0 0.0 200.0 0.15 0.0 0.0 0.0 0.0 0.0 0.0050 /
!0.654  10 20 105 1.387 0.0 0.0 0.0 203.0 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /
0.072 0 0 0 1.387 0.15 /
!
!**************************** L0A begin **************************
0.026 0 0 0 1.459 0.15 /
ERROR: ele_v_function not yet implemented for type:  wakefield
None
!to get 63.6MeV
0.052464 0 0 105 1.485 25905612.0 2856000000.0 258.5 4 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /
2.937928 0 0 105 1.537464 29913224.7 2856000000.0 288.5 5 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /
2.937928 0 0 105 1.537464 29913224.7 2856000000.0 348.5 6 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /
0.052464 0 0 105 4.475392 25905612.0 2856000000.0 258.5 7 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /
0.026 0 0 0 4.527856 0.15 /
!********************

In [17]:
# Nice printing
for e in eles:
    if e['type'] == 'spacecharge':
        print(e)

{'original': '0.0       1  1 -8  31.273    0.0     31.273 /', 'L': 0.0, 'itype': -8, 'nseg': 1, 'bmpstp': 1, 'type': 'spacecharge', 's': 53.148271840000014, 'is_on': 'False', 'name': 'spacecharge_1'}
{'original': '0.0       1  1  -8  38.16      1.0     38.16 /', 'L': 0.0, 'itype': -8, 'nseg': 1, 'bmpstp': 1, 'type': 'spacecharge', 's': 60.064309840000014, 'is_on': 'True', 'name': 'spacecharge_2'}
